# ML-Agents run with Stable Baselines 3
<img src="https://github.com/Unity-Technologies/ml-agents/blob/release_19_docs/docs/images/image-banner.png?raw=true" align="middle" width="435"/>

## Setup

In [ ]:
#@title Install Rendering Dependencies { display-mode: "form" }
#@markdown (You only need to run this code when using Colab's hosted runtime)

import os
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

pro_bar = display(progress(0, 100), display_id=True)

try:
  import google.colab
  INSTALL_XVFB = True
except ImportError:
  INSTALL_XVFB = 'COLAB_ALWAYS_INSTALL_XVFB' in os.environ

if INSTALL_XVFB:
  with open('frame-buffer', 'w') as writefile:
    writefile.write("""#taken from https://gist.github.com/jterrace/2911875
XVFB=/usr/bin/Xvfb
XVFBARGS=":1 -screen 0 1024x768x24 -ac +extension GLX +render -noreset"
PIDFILE=./frame-buffer.pid
case "$1" in
  start)
    echo -n "Starting virtual X frame buffer: Xvfb"
    /sbin/start-stop-daemon --start --quiet --pidfile $PIDFILE --make-pidfile --background --exec $XVFB -- $XVFBARGS
    echo "."
    ;;
  stop)
    echo -n "Stopping virtual X frame buffer: Xvfb"
    /sbin/start-stop-daemon --stop --quiet --pidfile $PIDFILE
    rm $PIDFILE
    echo "."
    ;;
  restart)
    $0 stop
    $0 start
    ;;
  *)
        echo "Usage: /etc/init.d/xvfb {start|stop|restart}"
        exit 1
esac
exit 0
    """)
  !sudo apt-get update
  pro_bar.update(progress(10, 100))
  !sudo DEBIAN_FRONTEND=noninteractive apt install -y daemon wget gdebi-core build-essential libfontenc1 libfreetype6 xorg-dev xorg
  pro_bar.update(progress(20, 100))
  !wget http://security.ubuntu.com/ubuntu/pool/main/libx/libxfont/libxfont1_1.5.1-1ubuntu0.16.04.4_amd64.deb 2>&1
  pro_bar.update(progress(30, 100))
  !wget --output-document xvfb.deb http://security.ubuntu.com/ubuntu/pool/universe/x/xorg-server/xvfb_1.18.4-0ubuntu0.12_amd64.deb 2>&1
  pro_bar.update(progress(40, 100))
  !sudo dpkg -i libxfont1_1.5.1-1ubuntu0.16.04.4_amd64.deb 2>&1
  pro_bar.update(progress(50, 100))
  !sudo dpkg -i xvfb.deb 2>&1
  pro_bar.update(progress(70, 100))
  !rm libxfont1_1.5.1-1ubuntu0.16.04.4_amd64.deb
  pro_bar.update(progress(80, 100))
  !rm xvfb.deb
  pro_bar.update(progress(90, 100))
  !bash frame-buffer start
  os.environ["DISPLAY"] = ":1"
pro_bar.update(progress(100, 100))

### Installing ml-agents

In [ ]:
try:
  import mlagents
  print("ml-agents already installed")
except ImportError:
  !python -m pip install -q mlagents==0.28.0
  print("Installed ml-agents")

## Run the Environment

### Import dependencies and set some high level parameters.

In [29]:
from math import ceil

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor

from mlagents_envs.envs.unity_vec_env import make_mla_sb3_env, LimitedConfig

# 250K should train to a reward ~= 0.90 for the "Basic" environment.
# We set the value lower here to demonstrate just a small amount of trianing.
TOTAL_TAINING_STEPS_GOAL = 40 * 1000
NUM_ENVS = 12
STEPS_PER_UPDATE = 2048

### Start Environment from the registry

In [ ]:
# -----------------
# This code is used to close an env that might not have been closed before
try:
  env.close()
except:
  pass
# -----------------

env = make_mla_sb3_env(
    config=LimitedConfig(
        env_path_or_name='Basic',  # Can use any name from a registry or a path to your own unity build.
        base_port=6006,
        base_seed=42,
        num_env=NUM_ENVS,
        allow_multiple_obs=True,
    ),
    no_graphics=True,  # Set to false if you are running locally and want to watch the environments move around as they train.
)

### Create the model

In [ ]:
# Helps gather stats for our eval() calls later so we can see reward stats.
env = VecMonitor(env)
# Attempt to approximate settings from 3DBall.yaml
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=lambda prog: 0.0003 * (1.0 - prog),
    # Uncomment this if you want to log tensorboard results when running this notebook locally.
    # tensorboard_log="results",
    n_steps=int(STEPS_PER_UPDATE),
)

### Train the model

In [ ]:
training_rounds = ceil(TOTAL_TAINING_STEPS_GOAL / int(STEPS_PER_UPDATE * NUM_ENVS))
for i in range(training_rounds):
    print(f"Training round {i + 1}/{training_rounds}")
    # NOTE: rest_num_timesteps should only happen the first time so that tensorboard logs are consistent.
    model.learn(total_timesteps=6000, reset_num_timesteps=(i == 0))
    model.policy.eval()

### Close the environment
Frees up the ports being used.

In [ ]:
env.close()
print("Closed environment")
